## Динамическая генерация тестов

+ Нужно сделать много однотипных тестов (одна и та же функция/метод и т.д.)
+ Отличаются только входное значение и ожидаемый результат
+ Лучше не писать кучу почти одинакового кода, а генерировать тесты динамически. 

In [1]:
def parse_name(name):
    parts = name.strip().split()
    surname, name, patr = '', '', ''
    if len(parts) == 1:
        name = parts[0]
    elif len(parts) == 2:
        surname, name = parts[0], parts[1]
    elif len(parts) == 3:
        surname, name, patr = parts
    elif len(parts) > 3:
        surname, name, patr = parts[0], ' '.join(parts[1:-1]), parts[-1]
    return surname, name, patr

In [2]:
import unittest
class ParseNameTestCase(unittest.TestCase):
    def test_one_word(self):
        self.assertEqual(('', 'Петр', ''), parse_name('Петр'))
    def test_two_words(self):
        self.assertEqual(('Петров', 'Петр', ''), parse_name('Петров Петр'))
    def test_three_words(self):
        self.assertEqual(('Петров', 'Петр', 'Петрович'), 
                         parse_name('Петров Петр Петрович'))
    def test_more_words(self):
        self.assertEqual(('Петрова', 'Анна Мария', 'Васильевна'), 
                         parse_name('Петрова Анна Мария Васильевна'))
    def test_no_words(self):
        self.assertEqual(('', '', ''), parse_name(''))
        
if __name__ == "__main__":
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

.....
----------------------------------------------------------------------
Ran 5 tests in 0.002s

OK


Cпособ 1 - просто в цикле:
+ все данные проверяются в одном тесте
+ метод прекращает работу после первой AssertionError
+ все что дальше не проверяется 

In [3]:
import unittest
class ParseNameTestCase(unittest.TestCase):
    def test_valid_name_parsing(self):
        for parsed_name, name in [
            (('', 'Петр', ''), ('Петр')),
            (('Петров', 'Петр', ''), ('Петров Петр')),
            (('Петров', 'Петр', 'Петрович'), ('Петров Петр Петрович')),
            (('Петрова', 'Анна Мария', 'Васильевна'), ('Петрова Анна Мария Васильевна')),
            (('', '', ''), (''))]:
            self.assertEqual(parsed_name, parse_name(name))
        
if __name__ == "__main__":
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


Cпособ 2 (он лучше) - c помощью пакета ***parametErized*** (не путать с parameTRized без e):
+ на каждую пару генерируется отдельный независимый тест
+ точно понятно, что именно пошло не так
+ один упавший тест не влияет на все остальные

In [4]:
!pip install parameterized
from parameterized import parameterized 

In [5]:
import unittest
class ParseNameTestCase(unittest.TestCase):
    @parameterized.expand(
        [(('', 'Петр', ''), ('Петр')),
         (('Петров', 'Петр', ''), ('Петров Петр')),
         (('Петров', 'Петр', 'Петрович'), ('Петров Петр Петрович')),
         (('Петрова', 'Анна Мария', 'Васильевна'), ('Петрова Анна Мария Васильевна')),
         (('', '', ''), (''))])
    def test_valid_name_parsing(self, parsed_name, name):
        self.assertEqual(parsed_name, parse_name(name))
        
if __name__ == "__main__":
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

.....
----------------------------------------------------------------------
Ran 5 tests in 0.002s

OK


**Задание**:
   + переписать тесты для parse_numeric с использованием parameterized

In [19]:
import re
import datetime
class RuDateParser:

    def parse_numeric(self, date): 
        """ 
        Парсит даты в формате dd-mm-yyyy

        >>> RuDateParser().parse_numeric('01-12-2010')
        datetime.datetime(2010, 12, 1, 0, 0)

        >>> RuDateParser().parse_numeric('01/12/2010')

        >>> RuDateParser().parse_numeric('не дата')
        """
        if not isinstance(date, str):
            raise TypeError
        # \d - digit, то же самое, что [0-9]
        elif not re.match('\d{2}-\d{2}-\d{4}', date): 
            return None
        date_splitted = date.split('-')
        day = int(date_splitted[0])
        month = int(date_splitted[1])
        year = int(date_splitted[2]) 
        try:
            return datetime.datetime(day=day, month=month, year=year)
        except ValueError:
            return None

In [22]:
import unittest

class RuDateParserTestCase(unittest.TestCase):
    
    def setUp(self):
        self.parser = RuDateParser()
        
    def tearDown(self):
        pass

    # тестируем поведение при правильных входных данных
    def test_parse_numeric_matching_string(self):
        self.assertEqual(datetime.datetime(day=12, month=1, year=2020), 
                         self.parser.parse_numeric('12-01-2020'))
        
    # тестируем поведение при вводе строки, не содержащей дату в нужном формате
    def test_parse_numeric_unmatching_string(self):
        self.assertEqual(None, self.parser.parse_numeric('12/01/2020'))
        
    def test_parse_numeric_day_too_large(self):
        self.assertEqual(None, self.parser.parse_numeric('52-01-2020'))
        
    def test_parse_numeric_month_too_large(self):
        self.assertEqual(None, self.parser.parse_numeric('21-21-2020'))
        
    def test_parse_numeric_day_zero(self):
        self.assertEqual(None, self.parser.parse_numeric('0-01-2020'))
        
    def test_parse_numeric_month_zero(self):
        self.assertEqual(None, self.parser.parse_numeric('01-00-2020'))
        
    # тестируем поведение при неправильном типе входных данных
    def test_parse_numeric_incorrect_input_type(self):
        self.assertRaises(TypeError, self.parser.parse_numeric, 123)

# запустить все тесты
if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False) # colab

.....s.......
----------------------------------------------------------------------
Ran 13 tests in 0.009s

OK (skipped=1)


## Что еще полезного есть в unittest

### Пропуск тестов
+ если какие-то тесты нужны/должны работать только при определенных условиях

In [21]:
# # это не рабочий код, он для примера и пояснения
class MyTestCase(unittest.TestCase):
    @unittest.skip("demonstrating skipping")
    def test_nothing(self):
        self.fail("shouldn't happen")

    @unittest.skipIf(mylib.__version__ < (1, 3),
                     "not supported in this library version")
    def test_format(self):
        # Tests that work for only a certain version of the library.
        pass

    @unittest.skipUnless(sys.platform.startswith("win"), "requires Windows")
    def test_windows_support(self):
        # windows specific testing code
        pass

    def test_maybe_skipped(self):
        if not external_resource_available():
            self.skipTest("external resource not available")
        # test code that depends on the external resource
        pass

In [23]:
import unittest
class ParseNameTestCase(unittest.TestCase):
    def test_one_word(self):
        self.assertEqual(('', 'Петр', ''), parse_name('Петр'))
    def test_two_words(self):
        self.assertEqual(('Петров', 'Петр', ''), parse_name('Петров Петр'))
    def test_three_words(self):
        self.assertEqual(('Петров', 'Петр', 'Петрович'), 
                         parse_name('Петров Петр Петрович'))
    def test_more_words(self):
        self.assertEqual(('Петрова', 'Анна Мария', 'Васильевна'), 
                         parse_name('Петрова Анна Мария Васильевна'))
    def test_no_words(self):
        self.assertEqual(('', '', ''), parse_name(''))
        
    @unittest.skip("this feature is not implemented yet")    
    def test_wrong_type(self):
        self.assertEqual(('', '', ''), parse_name(None))
        
if __name__ == "__main__":
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

.....s.......
----------------------------------------------------------------------
Ran 13 tests in 0.013s

OK (skipped=1)


## Структура проекта c тестами

+ все тесты в отдельной папке, название test или tests
+ лучше превратить в пакет - добавить \_\_init\_\_.py (можно пустой)
+ названия файлов с тестами начинаются с test_

```
├── project root directory      
   ├── main project directory
   │   ├── ...
   │   ├── ...
   │    
   └── tests
       ├── __init__.py
       ├── test_*.py
       └── test_*.py 

```

## Запуск тестов 

### командная строка

+ все тесты в проекте: ```python -m unittest discover ```
+ все тесты в модуле: ``` python -m unittest tests.test_something ```
+ все тесты в классе (тест-кейсе): 
``` python -m unittest tests.test_something.SomeTestCase ```


Флаг ***-m*** означает, что интерпретор найдет модуль/пакет с нужным именем и запустит его как  [***\_\_main\_\_***](https://docs.python.org/3/library/__main__.html#module-__main__), расширение (***.py***) писать не нужно ([документация](https://docs.python.org/3/using/cmdline.html#cmdoption-m)).

### PyCharm

+ правой кнопкой мыши на папку/файл/класс, выбрать run 'Unittest in ...'

**Задание**
+ склонировать/скачать этот репозиторий https://github.com/clips/pattern
+ ```pip install feedparser cherrypy```
+ запустить все тесты в PyCharm, сделать так, чтобы они запустились
+ посмотреть, что будет (в идеале вы должны увидеть пройденные, непройденные и пропущенные тесты)
+ понять, почему не проходятся тесты test_web.TestDocumentParser, сделать так, чтобы проходились (если у вас проходятся, то помочь товарищу)